<a href="https://colab.research.google.com/github/Myrto-Iglezou/AI2-project3/blob/master/Question-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## YΣ19 Artificial Intelligence II
# Homework 3

### Iglezou Myrto - 111520170038

# Question 1

In [2]:
import pandas as pd 
from pandas import DataFrame, read_csv

Download the dataset.

In [3]:
import gdown
url = 'https://drive.google.com/file/d/1dTIWNpjlrnTQBIQtaGOh0jCRYZiAQO79/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
output = "twitterData.csv"
gdown.download(path, output, quiet=True)
twitterData = pd.read_csv("/content/twitterData.csv",nrows=20000)
df = pd.DataFrame(data=twitterData)
df.head(5)

,Unnamed: 0,target,id,date,flag,user,text
0,680949,0,2249621587,Fri Jun 19 22:41:08 PDT 2009,NO_QUERY,sukumarpant,#brokenpromises...
1,406741,0,2059003515,Sat Jun 06 16:03:21 PDT 2009,NO_QUERY,MTMSparrow,David Carradine so sad. Thai's law not sure i...
2,1337108,4,2017466467,Wed Jun 03 08:26:14 PDT 2009,NO_QUERY,itsmemcee,A @ 415 B @ 425. Tell your bro i say congrats!
3,1560887,4,2186457254,Mon Jun 15 18:52:04 PDT 2009,NO_QUERY,jdfreivald,@littlefluffycat Indeed.
4,1466295,4,2064458395,Sun Jun 07 06:19:20 PDT 2009,NO_QUERY,CrazyHan,Completed Race 4 Life in 58mins with girlies f...


Creating a new dataframe with only two columns, target and text.


In [4]:
df.drop(df.columns.difference(['text','target']), 1, inplace=True)
df['target'].replace({4: 1}, inplace = True)
df.head(5)

,target,text
0,0,#brokenpromises...
1,0,David Carradine so sad. Thai's law not sure i...
2,1,A @ 415 B @ 425. Tell your bro i say congrats!
3,1,@littlefluffycat Indeed.
4,1,Completed Race 4 Life in 58mins with girlies f...


## Data pre-processing

### Removing the scpecial characters

In [5]:
import re

def RemoveSpecialCharacters(x):
    x = re.sub(r'[^\w]+', " ", x)
    return x

In [6]:
df['text'] = df['text'].apply(lambda x: RemoveSpecialCharacters(x) )

### Lemmatize

In [7]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

df['text'] = df['text'].apply(lambda x: WordNetLemmatizer().lemmatize(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Removing the upercase

In [8]:
df['text'] = df['text'].apply(lambda x: x.lower())

In [9]:
df.head(5)

,target,text
0,0,brokenpromises
1,0,david carradine so sad thai s law not sure if ...
2,1,a 415 b 425 tell your bro i say congrats
3,1,littlefluffycat indeed
4,1,completed race 4 life in 58mins with girlies f...


## Word embedding - GloVe
GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space. 

In [10]:
from torchtext.vocab import GloVe
glove = GloVe(name='twitter.27B', dim=25)

.vector_cache/glove.twitter.27B.zip: 1.52GB [12:03, 2.10MB/s]                            
100%|█████████▉| 1190227/1193514 [00:35<00:00, 33874.78it/s]

In [11]:
from torchtext.data import Field
text_field = Field(sequential=True,tokenize='spacy',fix_length=100)

label_field = Field(sequential=False, use_vocab=False)
preprocessed_text = df['text'].apply(lambda x: text_field.preprocess(x)) #apply preprocess manually
text_field.build_vocab(preprocessed_text, vectors=glove)  # load embedding with 25d
vocab = text_field.vocab  # get the vocab instance

### Create the datasets

We have train, validation and test datasets that contain text and label information. Train has 80%, validation 10% and test 10%.

In [12]:
from torchtext.data import Dataset, Example
fields = (('target', label_field),('text', text_field))
train_dataset, valid_dataset, test_dataset= Dataset(examples=[Example.fromlist(list(r), fields)
                                               for i, r in df.iterrows()],
                                               fields = fields).split(split_ratio = [0.8, 0.1, 0.1])

## Batching sentences together 
Use *BucketIterator* to create the batches for the 3 datasets with batch size 64.

In [13]:
from torchtext.data import BucketIterator
train_iter, valid_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, valid_dataset, test_dataset),
                                                          batch_sizes=(64,64,64), sort=False)

## Training on GPU

In [14]:
import torch
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


## Architecture of RNN

In [15]:
import torch.nn as nn
import numpy as np
from numpy import zeros

In [24]:
class RNNclassifier(nn.Module):
    """
    Wrapper module for RNN, LSTM, GRU cells using pytorch api. 
    The output is fed to a FFN for classification.
    """
    def __init__(self,embedding, cell_type, input_size, output_size,embedding_dim = 25, hidden_size=100, num_layers=1, bidirectional=False):
        super(RNNclassifier, self).__init__()
        
        cells = {
        "RNN" : nn.RNN,
        "LSTM"    : nn.LSTM,
        "GRU"     : nn.GRU
    }
        # Initializations
        self.embedding = embedding
        # dropout layer
        self.dropout = nn.Dropout(0.2)
        self.cell_type = cell_type
        self.input_size, self.hidden_size, self.num_layers, self.output_size = input_size, hidden_size, num_layers, output_size
        self.bidirectional = bidirectional
        self.sigmoid = nn.Sigmoid()
        self.rnn = cells[cell_type](         # Pick the specific model
            input_size=embedding_dim,           # Number of features for each time step
            hidden_size=hidden_size,         # Rnn hidden embedding representation
            num_layers=num_layers,           # Number of layers (if >1 it is a stacked RNN)
            dropout=0.2,
            bidirectional=bidirectional, # If true it is a 
            batch_first=True,       # input & output will have batch size as 1st dimension. e.g. (batch, time_step, input_size)
        )
        
        if bidirectional: # we'll have 2 more layers as the output to the final layer
            self.out = nn.Linear(hidden_size * 2, output_size)
        else:
            self.out = nn.Linear(hidden_size, output_size)
            
    def forward(self, x, prints=False):

        x = self.embedding[x]
        # x = self.dropout(x)

        if prints: 
            print('images shape:', x.shape)


        # LSTM cells reutrn also the last cell states
        if self.cell_type == 'LSTM':
            r_out, (last_hidden_state, last_cell_state) = self.rnn(x.detach()) 
        else:
            r_out, last_hidden_state = self.rnn(x.detach())
      
        r_out = r_out[:, -1, :]

        out = self.out(r_out)

        out = self.dropout(out)

        return self.sigmoid(out)

In [17]:
def train(model,X,y,optimizer,loss_fn):

    if train_on_gpu:
      X, y = X.cuda(), y.cuda()

# step 1. Zero the gradients
    optimizer.zero_grad()
# step 2. Compute the output
    y_pred = model(X)

    if train_on_gpu:
      y_pred = y_pred.squeeze().cuda()
    else:
      y_pred = y_pred.squeeze()
# step 3. Compute the loss
    loss = loss_fn(y_pred, y)
# step 4. use loss to produce gradients
    loss.backward()
# step 5. use optimizer to take gradient step
    optimizer.step()

    return loss.item(),y_pred

In [18]:
def validate(classifier,X,y,loss_fn):
  y_pred =  classifier(X)

  if train_on_gpu:
    y_pred = y_pred.squeeze().cuda()
  else:
    y_pred = y_pred.squeeze()

  loss = loss_fn(y_pred, y)

  return loss.item(),y_pred


In [19]:
def predict(model,X):
    outputs = model(X)
    return torch.round(outputs)

## Training



In [26]:
# ====== STATICS ======
input_size = 100       # length is 100, given in Field
hidden_size = 200     # number of hidden neurons
num_layers = 2        # number of layers
output_size = 1     
bidirectional = True
cell_type = 'GRU'
embedding_dim = 25
# =====================

net = RNNclassifier(vocab.vectors,cell_type, input_size, output_size,embedding_dim, hidden_size, num_layers, bidirectional)
print(net)

RNNclassifier(
  (dropout): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
  (rnn): GRU(25, 200, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (out): Linear(in_features=400, out_features=1, bias=True)
)


In [27]:
EPOCH = 10
LR = 0.01

optimizer = torch.optim.Adam(net.parameters(), lr=LR)   # optimize all net parameters
loss_function = nn.BCELoss() 

In [28]:
train_on_gpu = False

In [29]:
from sklearn.metrics import accuracy_score, f1_score,recall_score,precision_score

# move model to GPU, if available
if train_on_gpu:
    net.cuda()

# Start train mode

# Accumulate losses
train_losses = []
train_accuracy = []
steps = []
valid_accuracy = []

# Evaluate the progress every # batches

# Iterate over the whole dataset for one epoch

for epoch in range(EPOCH):
    net.train() 
    for batch in train_iter:

      loss, pred = train(net, batch.text.T, batch.target.float(), optimizer, loss_function)
      train_losses.append(loss)
      
      pred = torch.round(pred)
      pred = pred.detach().numpy()
      test = batch.target.float()
      test = test.numpy()

      train_accuracy.append(accuracy_score(test, pred))

      # print(f'Epoch: {epoch}|  Train CE loss: {loss:4f} | Test accuracy: {100 * train_accuracy[-1]:.2f} %')
      
    #Clip gradients
      for param in net.parameters():
          if param.grad is None:
              continue
          grad_val = torch.clamp(param.grad, -5, 5)
      # -----------------------------------
    
    net.eval()
    print("\n--------------------------------------------------------------------\n")
    for step,batch in enumerate(valid_iter):
        with torch.no_grad():
          # Faster inference
            if(train_on_gpu):
                  inputs, labels = batch.text.cuda(), batch.target.float().cuda()
            else:
              inputs, labels = batch.text, batch.target.float()
            
            loss, pred = validate(net, inputs.T, labels,loss_function)

            pred = torch.round(pred)
            pred = pred.detach().numpy()
            valid = labels
            valid = valid.numpy()

            valid_accuracy.append(accuracy_score(valid, pred))
            # print(f'Epoch: {epoch}| Validation CE loss: {loss:4f} | Test accuracy: {100 * valid_accuracy[-1]:.2f} %')

    print(f'Epoch: {epoch}| | Train accuracy: {100 * np.mean(train_accuracy):.2f} %')
    print(f'Epoch: {epoch}| | Valid accuracy: {100 * np.mean(valid_accuracy):.2f} %')
   

# # Plot results
# plt.figure(figsize=(12,5))
# plt.plot(train_accuracy, label='Train acc.')
# plt.plot(steps, valid_accuracy, label='Test acc.', marker='o')
# plt.xlabel('# of batch steps', fontsize=12)
# plt.ylabel('Performance', fontsize=12)
# plt.legend()
# plt.show()


--------------------------------------------------------------------

Epoch: 0| | Train accuracy: 50.74 %
Epoch: 0| | Valid accuracy: 57.37 %

--------------------------------------------------------------------

Epoch: 1| | Train accuracy: 52.06 %
Epoch: 1| | Valid accuracy: 57.10 %


KeyboardInterrupt: ignored

## Check for Overfitting-Underfitting

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))

plt.plot(valid_losses,'o-',color="r")

plt.plot(list_of_losses,'o-',color="g")

plt.grid()

plt.plot(valid_losses, 'o-', color="r",
                label="Validation loss")
plt.plot(list_of_losses,  'o-', color="g",
                label="Training loss")

plt.legend(loc="best")

plt.xlabel("epochs")
plt.ylabel("losses")
plt.title("Scalability of the model")

plt.show()

## Testing

In [ ]:
from sklearn.metrics import accuracy_score, f1_score,recall_score,precision_score

accuracy = []
f1 = []
recall = []
precision = []

for batch in test_iter:
      
    y_pred = predict(net,batch.text.T)
    y_pred = y_pred.detach().numpy()
    y_test = batch.target.float()
    y_test = y_test.numpy()

    acc = accuracy_score(y_test, y_pred)
    f1_sc = f1_score(y_test, y_pred, average='macro')
    rec = recall_score(y_test, y_pred, average='macro')
    prec = precision_score(y_test, y_pred, average='macro')
    
    accuracy.append(acc)
    f1.append(f1_sc)
    recall.append(rec)
    precision.append(prec)
    

## Scores

In [ ]:
print("Accuracy: ",np.mean(accuracy))
print("F1-score: ",np.mean(f1))
print("Recall-score: ",np.mean(recall))
print("Precision: ",np.mean(precision))

# **Remarks**



*   For this model we used GloVe for the pre-process, with vector size 25. RMS was chosen as an optimizer and BCELoss as loss function, because of the binary classification.
*   For neural network architecture:

> * The first layer is an Embedding layer, which matches the input vectors to the vocabulary vectors.
> * The second layer is a Linear hidden layer,which accepts as input size, the size of the features multiplied by the size of the GloVe vectors. It uses ReLU as activation function.
>* The output layer is Linear with input dimension  200 and output 1 with sigmoid as activation function.









## References



*   https://towardsdatascience.com/deep-learning-for-nlp-with-pytorch-and-torchtext-4f92d69052f


